In [1]:
import os
from pathlib import Path


FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day16.txt'

with open(FOLDER / in_file) as f:
    data = f.read().strip()

In [2]:
from functools import reduce 

def process_results(subs, i):
    procs = {
        0: sum,
        1: lambda l: reduce(lambda a, n: a * n,l, 1),
        2: min,
        3: max,
        5: lambda l: int(l[0] > l[1]),
        6: lambda l: int(l[0] < l[1]),
        7: lambda l: int(l[0] == l[1]),
    }
    return procs[i](subs)

class Bitreader:
    def __init__(self, s, stats):
        self.stats = stats
        self.s = s
        self.i = 0
        
    def read(self, n):
        j = self.i
        self.i += n
        return self.s[j:n+j]

    def remaining(self):
        return len(self.s) - self.i > 0

    def operator(self, NODE_TYPE):
        type_id = self.read(1)
        results = []

        if type_id == '0':
            length = int(self.read(15),2)
            subs = self.read(length)   
            sub_reader = Bitreader(subs, stats)

            while sub_reader.remaining():
                results.append(sub_reader.packet_dispatch())

        elif type_id == '1':
            length = int(self.read(11),2)
            for _ in range(length):
                results.append(self.packet_dispatch())

        return process_results(results, NODE_TYPE)


    def literal(self, NODE_TYPE):
        group = self.read(5)
        acc = group[1:]

        while group[0]  != '0':
            group = self.read(5)
            acc += group[1:]
        return int(acc, 2)


    def packet_dispatch(self):
        types = { 4: self.literal }   
        version = int(self.read(3), 2)

        self.stats['versions'] += version
        packet_type = int(self.read(3), 2)
        packet_class = types.get(packet_type, self.operator)
        return packet_class(packet_type)
  

In [3]:
lookup = {
    '0': '0000',
    '1': '0001',
    '2': '0010',
    '3': '0011',
    '4': '0100',
    '5': '0101',
    '6': '0110',
    '7': '0111',
    '8': '1000',
    '9': '1001',
    'A': '1010',
    'B': '1011',
    'C': '1100',
    'D': '1101',
    'E': '1110',
    'F': '1111'
}

 
def hex_to_b_string(hex_str):
    return ''.join(lookup[char] for char in hex_str)

s = hex_to_b_string(data)
stats = {'versions': 0}
b = Bitreader(s, stats)

solution_2 = b.packet_dispatch()
solution_1 = stats['versions']

In [4]:
print(f"Solution 1:", solution_1)
print(f"Solution 2:", solution_2)

Solution 1: 847
Solution 2: 333794664059
